# Мастерская (EDA+дашборд)
## Анализ вакансий для Data Analyst или BI analyst entry level в ЕU с LinkedIn.

## Цель: 
Визуализировать информацию о рынке вакансий для дата аналитика или BI аналитика в Европе
## Источник данных: 
файл сsv c вакансиями за неделю, спарсенными с LinkedIn 08/09/2022
## ТЗ
1. Распарсить предоставленный csv файл с помощью BS 4, создав следующие признаки:
- наименование вакансии
- город
- страна
- тип занятости (online, hybride, on-site)
- компания
- размер компании (количество работников)
- сфера деятельности компании
- требуемые хард скилы
- дата публикации вакансии
- количество кандидатов на вакансию

2. Подготовка данных к визуализации:
- фильтрация датафрейма с оставлением только вакансий для аналитиков данных и BI аналитиков
- удаление дубликатов
- удаление ненужных атрибутов (признаков)

3. Визуализация данных в Tableau

In [1]:
import os
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np

import requests
from geopy.geocoders import Nominatim
import datetime as dt
from datetime import date, datetime, timedelta
#from datetime import datetime

from IPython.display import HTML

In [2]:
# Откроем файл для аналитиков
def path_open(name, pth1, pth2):
    if os.path.exists(pth1):
        name = pd.read_csv(pth1)
    elif os.path.exists(pth2):
        name = pd.read_csv(pth2)
    else:
        print('Something is wrong')
        
path_open('df', 'masterskaya_yandex_2022_09_07.csv', 
          'D:\IT\courses Python\yandex_practicum\\13 parsing\masterskaya_yandex_2022_09_07.csv'
         )
df = pd.read_csv('masterskaya_yandex_2022_09_07.csv', usecols=['html'])
df.info()
df.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 772 entries, 0 to 771
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   html    772 non-null    object
dtypes: object(1)
memory usage: 6.2+ KB


,html
0,"\n <div>\n <div class=""\n jobs-deta..."
1,"\n <div>\n <div class=""\n jobs-deta..."


## Наименование вакансии

In [3]:
# Выделим название вакансии
df['title'] = df['html'].apply(lambda x:  BeautifulSoup(x).find('h2').text.strip())
df.head(2)

# Посмотрим какие есть вакансии
#df['title'].unique()

,html,title
0,"\n <div>\n <div class=""\n jobs-deta...",Stage - Assistant Ingénieur Qualité - Beyrand ...
1,"\n <div>\n <div class=""\n jobs-deta...","développeur matlab/simulink, secteur automobil..."


In [4]:
# Отсортируем вакансии для дата аналитиков 
df_sorted = (
    df.loc[df['title']
    .str.contains('Analyst|Business Intelligence', case=False) == True]
    .reset_index(drop=True)
)
print('Количество релевантных вакансий: {} шт, что составляет - {}%'.format(len(df_sorted), round(len(df_sorted)/ len(df) * 100)))

Количество релевантных вакансий: 362 шт, что составляет - 47%


In [5]:
# Достанем ссылки на вакансии. В html любая ссылка имеет аттрибут href
# Добавим спереди ресурс откуда 'https://linkedin.com' и закинем в лямбду.
df_sorted['link'] = df_sorted['html'].apply(lambda x: "https://linkedin.com" + BeautifulSoup(x).find('a').get('href'))
df_sorted[['link']].to_html(render_links=True, escape=False)
df_sorted.sample(2)

,html,title,link
252,"\n <div>\n <div class=""\n jobs-deta...",Data Analyst & BusinessObjects Developer,https://linkedin.com/jobs/view/3199645530/?alt...
291,"\n <div>\n <div class=""\n jobs-deta...",Data Analyst,https://linkedin.com/jobs/view/3249116636/?alt...


In [6]:
# Сделаем кликабельные ссылки в столбце link
def make_clickable(val):
    return f'<a href="{val}">{val}</a>'

df_sorted.style.format(make_clickable)
df_sorted.sample(2)

,html,title,link
308,"\n <div>\n <div class=""\n jobs-deta...",Inventory data analyst til Europas største kon...,https://linkedin.com/jobs/view/3247021945/?alt...
129,"\n <div>\n <div class=""\n jobs-deta...",Data Analyst,https://linkedin.com/jobs/view/3247043224/?alt...


In [7]:
# Нужно вытащить адрес
def get_geo(cell):
    try:
        return BeautifulSoup(cell).find('span', class_ = 'jobs-unified-top-card__bullet').text.strip()
    except:
        return np.nan
df_sorted['geo'] = df_sorted['html'].apply(get_geo)
df_sorted.sample(2)

,html,title,link,geo
163,"\n <div>\n <div class=""\n jobs-deta...",(Junior) H2 Modelling Analyst m/f/t,https://linkedin.com/jobs/view/3250680016/?alt...,"Essen, North Rhine-Westphalia, Germany"
106,"\n <div>\n <div class=""\n jobs-deta...",Data Analyst (m/w/d),https://linkedin.com/jobs/view/3248338605/?alt...,"Bonn, North Rhine-Westphalia, Germany"


## Город - city

In [8]:
# Функция будет создавать один признак с городом, другой - со страной. Получим город
def get_city(cell):
    try:
        if len(cell.split(',')) > 1:
            return cell.split(',')[0].strip()
        elif 'Greater' in cell:
            return cell.split()[1].strip()
        elif 'Area' in cell:
            return cell.split()[0].strip()
    except:
        return 'unknown'
df_sorted['city'] = df_sorted['geo'].apply(get_city)
df_sorted.head(2)

,html,title,link,geo,city
0,"\n <div>\n <div class=""\n jobs-deta...",Online Data Analyst,https://linkedin.com/jobs/view/3248499929/?alt...,"Skara, Vastra Gotaland County, Sweden",Skara
1,"\n <div>\n <div class=""\n jobs-deta...",Online Data Analyst - Belgium,https://linkedin.com/jobs/view/3248879065/?alt...,"West Flanders, Flemish Region, Belgium",West Flanders


## Страна - country

In [9]:
# Получим страну
def get_country(cell):
    try:
        if len(cell.split(',')) > 1:
            return cell.split(',')[-1].strip()
        else:
            return cell.split(',')[0].strip()
    except:
        return np.NaN
df_sorted['country'] = df_sorted['geo'].apply(get_country)
df_sorted.sample(2)

,html,title,link,geo,city,country
25,"\n <div>\n <div class=""\n jobs-deta...",Data Analyst (IT),https://linkedin.com/jobs/view/3254949074/?alt...,"Basel, Basel, Switzerland",Basel,Switzerland
300,"\n <div>\n <div class=""\n jobs-deta...",Junior Analyst (J12998),https://linkedin.com/jobs/view/3248176207/?alt...,"Tavira, Faro, Portugal",Tavira,Portugal


In [10]:
# Посмотрим пропуски и уникальные имена стран
for column in df_sorted:  
    row = df_sorted[column].isna().sum()
    print('{} : {} / {:.0%}'.format(column, row, row / len(df_sorted)))
    
print('---------------------------')
print('Уникальные имена стран:\n{}'.format(df_sorted['country'].unique()))

html : 0 / 0%
title : 0 / 0%
link : 0 / 0%
geo : 0 / 0%
city : 24 / 7%
country : 0 / 0%
---------------------------
Уникальные имена стран:
['Sweden' 'Belgium' 'Germany' 'France' 'Hungary'
 'Przemyśl Metropolitan Area' 'Gibraltar' 'Poland'
 'Grudziadz Metropolitan Area' 'Greater Norrköping Metropolitan Area'
 'Radom Metropolitan Area' 'Zamosc Metropolitan Area'
 'Greater Edinburgh Area' 'United Kingdom' 'Finland' 'Switzerland'
 'Netherlands' 'Lodz Metropolitan Area' 'Norway' 'Italy' 'Czechia'
 'Bulgaria' 'Slovakia' 'Portugal' 'Spain' 'Mons Metropolitan Area'
 'Bucharest Metropolitan Area' 'Greater Milan Metropolitan Area'
 'Greater Bologna Metropolitan Area' 'Luxembourg'
 'Ghent Metropolitan Area' 'Ireland' 'Bruges Metropolitan Area' 'Croatia'
 'Greater Gijón Metropolitan Area' 'Kortrijk Metropolitan Area' 'Slovenia'
 'Romania' 'Denmark' 'Greater Rome Metropolitan Area'
 'Greater Zurich Area' 'Namur Metropolitan Area'
 'Greater Barcelona Metropolitan Area' 'Greater Turin Metropolitan A

- В странах присутсвуют приставки областей, районов.
- Пропуски городов обработаем заменой

In [11]:
# Уберем пропуски городов
df_sorted['city'].fillna('unknown', inplace=True)

In [12]:
# Чтобы получить названия стран по названиям местности, воспользуемся сервисом geonames
def get_country_from(cell):
    if 'Greater' in cell:
        city_name = cell.split()[1].strip()
        response = requests.request("GET", f"https://www.geonames.org/search.html?q={city_name}&country=")
        country = BeautifulSoup(response.text).find_all('a')[9].text
        if country:
            return country
        else: 
            return cell
    elif 'Area' in cell:
        city_name = cell.split()[0].strip()
        response = requests.request("GET", f"https://www.geonames.org/search.html?q={city_name}&country=")
        country = BeautifulSoup(response.text).find_all('a')[9].text
        if country:
            return country
        else:
            return cell
    else:
        return cell

df_sorted['country'] = df_sorted['country'].apply(get_country_from)
df_sorted['country'].unique()

array(['Sweden', 'Belgium', 'Germany', 'France', 'Hungary',
       'Przemyśl Metropolitan Area', 'Gibraltar', 'Poland',
       'Radom Metropolitan Area', 'United Kingdom', 'Finland',
       'Switzerland', 'Netherlands', 'Norway', 'Italy', 'Czechia',
       'Bulgaria', 'Slovakia', 'Portugal', 'Spain', 'Romania',
       'Luxembourg', 'Ireland', 'Croatia', 'Slovenia', 'Denmark',
       'Austria', 'Lithuania', 'Greece'], dtype=object)

In [13]:
# Заменим то, что не сработало
df_sorted.loc[df_sorted['country'] =='Przemyśl Metropolitan Area', 'country']= 'Poland'
df_sorted.loc[df_sorted['country'] =='Radom Metropolitan Area', 'country']= 'Poland'

In [14]:
# Для дальнейшей визуализации потребуются географические координаты. Выделим их.
geolocator = Nominatim(user_agent = 'your_app')

# Широта
def get_latitude(cell):
    try:
        location = geolocator.geocode(cell)
        return location.latitude
    except:
        return np.nan

# Долгота
def get_longitude(cell):
    try:
        location = geolocator.geocode(cell)
        return location.longitude
    except:
        return np.nan

df_sorted['latitude'] = df_sorted['geo'].apply(get_latitude)
df_sorted['longitude'] = df_sorted['geo'].apply(get_longitude)

In [15]:
# Есть ли пропуск
print('Количество пропусков в долготе/ширине:', df_sorted['latitude'].isna().sum())
print('---------------------------')
df_sorted[df_sorted['latitude'].isna()].head(2)

Количество пропусков в долготе/ширине: 31
---------------------------


,html,title,link,geo,city,country,latitude,longitude
6,"\n <div>\n <div class=""\n jobs-deta...",Data Analyst,https://linkedin.com/jobs/view/3257247449/?alt...,Przemyśl Metropolitan Area,Przemyśl,Poland,NaN,NaN
14,"\n <div>\n <div class=""\n jobs-deta...",Data Analyst,https://linkedin.com/jobs/view/3257240118/?alt...,Grudziadz Metropolitan Area,Grudziadz,Poland,NaN,NaN


Пропуски- это области, регионы в geo. Город, мы выделяли из названия области.

Выделим количество кандидатов- candidats, название компании- company, тип занятости- employment.

In [16]:
# Выберем элементы из карточки компании
def span_element(df, class_name):
    soup = BeautifulSoup(df, 'lxml')
    general_info = soup.find_all('div', class_='jobs-unified-top-card__content--two-pane')
    for row in general_info:
      element = [element.text.strip() for element in row.find_all('span', class_=class_name, limit=1)]
      element = ''.join(element)
      return element

## Количество кандидатов - candidats

In [17]:
# Выделим количество кандидатов- candidats
df_sorted['candidats'] = df_sorted['html'].apply(lambda x: span_element(x, class_name='jobs-unified-top-card__applicant-count'))
df_sorted['candidats'].unique()

array(['12 applicants', '11 applicants', '2 applicants', '4 applicants',
       '25 applicants', '', '7 applicants', '14 applicants',
       '30 applicants', '26 applicants', '10 applicants', '9 applicants',
       '3 applicants', '23 applicants', '5 applicants', '49 applicants',
       '13 applicants', '31 applicants', '44 applicants', '15 applicants',
       '21 applicants', '6 applicants', '33 applicants', '29 applicants',
       '17 applicants', '8 applicants', '62 applicants', '50 applicants',
       '32 applicants', '36 applicants', '65 applicants', '1 applicant',
       '20 applicants', '28 applicants', '63 applicants', '75 applicants',
       '38 applicants', '24 applicants', '85 applicants', '22 applicants',
       '57 applicants', '79 applicants', '107 applicants',
       '42 applicants', '40 applicants', '73 applicants',
       '167 applicants', '78 applicants', '70 applicants',
       '16 applicants', '67 applicants', '56 applicants', '35 applicants',
       '60 applicants'

In [18]:
# Заменим строчный пропуск
df_sorted.loc[df_sorted['candidats'] == '', 'candidats'] = None

# Уберем слово "applicants" и преобразуем тип.
df_sorted['candidats'] = df_sorted['candidats'].str.replace(r'\D', '', regex=True).astype('Int64')
df_sorted['candidats'].fillna(0, inplace=True)
df_sorted['candidats'].unique()

<IntegerArray>
[ 12,  11,   2,   4,  25,   0,   7,  14,  30,  26,  10,   9,   3,  23,   5,
  49,  13,  31,  44,  15,  21,   6,  33,  29,  17,   8,  62,  50,  32,  36,
  65,   1,  20,  28,  63,  75,  38,  24,  85,  22,  57,  79, 107,  42,  40,
  73, 167,  78,  70,  16,  67,  56,  35,  60,  48,  19,  76,  41,  18, 108,
 144,  55,  98,  92,  34, 109, 111,  45,  88,  93,  43,  27,  61, 147,  52,
 137,  54,  66,  37,  94, 148, 169,  71, 110]
Length: 84, dtype: Int64

У нас есть пропуски в данных- количество кандидатов. Это может быть обусловлено свежестью размещения вакансии, на которую еще никто не успел откликнуться.    
А чтобы привести тип данных к целочисленному, даже если в столбце есть пропуски испльзуем astype('Int64')

## Тип занятости - employment

In [19]:
# Выделим тип занятости- employment
df_sorted['employment'] = df_sorted['html'].apply(lambda x: span_element(x, class_name='jobs-unified-top-card__workplace-type'))
df_sorted['employment'].unique()

array(['Remote', 'On-site', 'Hybrid', ''], dtype=object)

In [20]:
# Заменим пропуски 
df_sorted['employment'].replace('', 'unknown', inplace=True)
df_sorted['employment'].fillna('unknown', inplace=True)
df_sorted['employment'].unique()

array(['Remote', 'On-site', 'Hybrid', 'unknown'], dtype=object)

## Название компании - company

In [21]:
# Выделим название компании- company
df_sorted['company'] = df_sorted['html'].apply(lambda x: span_element(x, class_name='jobs-unified-top-card__company-name'))
print('Количество дубликатов в столбце company:', df_sorted['company'].duplicated().sum())
print('---------------------------')
print('Количество пропусков в столбце company:', df_sorted['company'].isna().sum())

Количество дубликатов в столбце company: 153
---------------------------
Количество пропусков в столбце company: 0


153 компании дублируются. Вероятно, у одной компании несколько вакансий.
## Дата публикации - publication date

In [22]:
# Выделим дату публикации- publication date до `2022-09-07 17:00:00`
parse_date_time = dt.datetime(2022, 9, 7, 17, 0, 0)
print(parse_date_time)


def get_date_time(cell):
    time_info = BeautifulSoup(cell).find('span', class_ = 'jobs-unified-top-card__posted-date').text.strip()
    time_unit_name = time_info.split(' ')[1]
    time_unit = int(time_info.split(' ')[0])
    if time_unit_name in ['seconds', 'second']:
        return parse_date_time - dt.timedelta(seconds=time_unit)
    elif time_unit_name in ['minutes', 'minute']:
        return parse_date_time - dt.timedelta(minutes=time_unit)
    elif time_unit_name in ['hours', 'hour']:
        return parse_date_time - dt.timedelta(hours=time_unit)
    elif time_unit_name in ['days', 'day']:
        return parse_date_time - dt.timedelta(days=time_unit)
    elif time_unit_name in ['weeks', 'week']:
        return parse_date_time - dt.timedelta(weeks=time_unit)
    else:
        return np.nan
df_sorted['publication_date'] = df_sorted['html'].apply(get_date_time)
df_sorted.sample(2)

2022-09-07 17:00:00


,html,title,link,geo,city,country,latitude,longitude,candidats,employment,company,publication_date
243,"\n <div>\n <div class=""\n jobs-deta...",Internet Analyst (Remote Set-up),https://linkedin.com/jobs/view/3249552448/?alt...,"Cracow, Małopolskie, Poland",Cracow,Poland,50.060674,19.937139,8,Remote,TELUS International AI Data Solutions,2022-09-02 17:00:00
216,"\n <div>\n <div class=""\n jobs-deta...",Junior Analyst (J12997),https://linkedin.com/jobs/view/3248177345/?alt...,France,unknown,France,46.603354,1.888334,60,Remote,Pareto,2022-09-02 17:00:00


Вариант обработки даты

if 'day' in string:
    days_ago = int(''.join([x for x in string if x.isdigit()]))
    return date_of_parsing - timedelta(days = days_ago)

elif 'hours' in string:
    hours_ago = int(''.join([x for x in string if x.isdigit()]))
    return date_of_parsing - timedelta(hours = hours_ago)
    
else:
    week_ago = int(''.join([x for x in string if x.isdigit()]))
    return date_of_parsing - timedelta(weeks = week_ago)

In [23]:
df_sorted['publication_date'].describe(datetime_is_numeric=True)

count                              362
mean     2022-09-04 19:16:52.707182592
min                2022-08-31 17:00:00
25%                2022-09-02 17:00:00
50%                2022-09-05 17:00:00
75%                2022-09-06 17:00:00
max                2022-09-07 16:50:00
Name: publication_date, dtype: object

In [24]:
# Выделим размер компании и сферу деятельности
def size_field_of_activity(cell):
    try:
        return BeautifulSoup(cell).find('div', class_ = 'mt5 mb2').find_all('li')[1].text.strip()
    except:
        return np.nan
df_sorted['size_field'] = df_sorted['html'].apply(size_field_of_activity)

# Посмотрим есть ли пропуск
df_sorted.query('size_field.isna()')

,html,title,link,geo,city,country,latitude,longitude,candidats,employment,company,publication_date,size_field
286,"\n <div>\n <div class=""\n jobs-deta...",Customer Ledger Data Analyst,https://linkedin.com/jobs/view/3254329032/?alt...,"Stockport, England, United Kingdom",Stockport,United Kingdom,53.407901,-2.160243,0,On-site,Birnbach Communications,2022-09-06 17:00:00,NaN


In [25]:
# Вытащим ссылку, посмотрим вакансию
df_sorted.iloc[286]['link']

'https://linkedin.com/jobs/view/3254329032/?alternateChannel=search&refId=hoEkNmylbSwHUlyIHRvzIQ%3D%3D&trackingId=SwUmzWKVnKM6fhDoWNcdtw%3D%3D&trk=d_flagship3_search_srp_jobs'

In [26]:
# Заполним пропуск
df_sorted['size_field'].fillna('1-10 employees · Public Relations and Communications Services', inplace=True)

# Проверим, остались ли пропуски
df_sorted['size_field'].isna().sum()

0

## Размер компании - company_size

In [27]:
# Выделим размер компании (количество работников)
def company_size(cell):
    if 'employees' in cell:
        cell = cell.partition(' employees')[0].replace(',', '')
        return cell 
    else:
        return 'unknown'
    
df_sorted['company_size'] = df_sorted['size_field'].apply(company_size)

print('Количество пропусков в столбце company_size:', df_sorted['company_size'].isna().sum())
print('---------------------------')
print('Уникальные значения в столбце company_size:\n', df_sorted['company_size'].unique())

Количество пропусков в столбце company_size: 0
---------------------------
Уникальные значения в столбце company_size:
 ['10001+' '201-500' '1001-5000' '51-200' '5001-10000' '501-1000' '11-50'
 '1-10' 'unknown']


На сайте нет инфомации для заполнения. Оставляем так

In [28]:
# Категоризуем компании по размеру
def size_category(col):
    if ('1-10' in col) | ('11-50' in col):
        return 'small'
    if ('51-200' in col) | ('201-500' in col):
        return 'medium'
    if ('10001+' in col) | ('1001-5000' in col) | (
            '5001-10000' in col) | ('501-1000' in col):
        return 'large'
    return 'unknown'
df_sorted['company_size_category'] = df_sorted['company_size'].apply(size_category)
df_sorted['company_size_category'].value_counts()

large      188
small       98
medium      65
unknown     11
Name: company_size_category, dtype: int64

## Сфера деятельности компании - field_of_activity

In [29]:
# Выделим сферу деятельности компании
def field_of_activity (cell):
    if 'Premium' in cell:
        return 'unknown'
    if ('employees' in cell) and ('·' not in cell):
        return 'unknown'
    if '·' in cell:
        cell = cell.partition('· ')[2]
        return cell
    else:
        return cell

df_sorted['field_of_activity'] = df_sorted['size_field'].apply(field_of_activity)

print('Количество пропусков в столбце field_of_activity:', 
df_sorted.loc[df_sorted['field_of_activity'] == 'unknown', 'field_of_activity'].count())


Количество пропусков в столбце field_of_activity: 18


In [30]:
# Сохраним файл в Tableau
df_sorted[['title', 'city', 'country', 'latitude', 'longitude']].to_csv('geo.csv', index = False)
df_sorted.to_csv('full_project.csv')
display(df_sorted.sample(2))
print('---------------------------')
print('Количество дубликатов:', df_sorted.duplicated().sum())
print('---------------------------')
df_sorted.info()

,html,title,link,geo,city,country,latitude,longitude,candidats,employment,company,publication_date,size_field,company_size,company_size_category,field_of_activity
165,"\n <div>\n <div class=""\n jobs-deta...",Data Analyst H/F,https://linkedin.com/jobs/view/3248992469/?alt...,"Hérault, Occitanie, France",Hérault,France,43.591422,3.355331,3,On-site,EPSI - L'école d'ingénierie informatique,2022-09-01 17:00:00,51-200 employees · Education Administration Pr...,51-200,medium,Education Administration Programs
137,"\n <div>\n <div class=""\n jobs-deta...",Data Analyst,https://linkedin.com/jobs/view/3223529005/?alt...,"Birmingham, England, United Kingdom",Birmingham,United Kingdom,52.479699,-1.902691,67,unknown,Balfour Beatty plc,2022-09-06 17:00:00,"10,001+ employees · Construction",10001+,large,Construction


---------------------------
Количество дубликатов: 0
---------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 362 entries, 0 to 361
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   html                   362 non-null    object        
 1   title                  362 non-null    object        
 2   link                   362 non-null    object        
 3   geo                    362 non-null    object        
 4   city                   362 non-null    object        
 5   country                362 non-null    object        
 6   latitude               331 non-null    float64       
 7   longitude              331 non-null    float64       
 8   candidats              362 non-null    Int64         
 9   employment             362 non-null    object        
 10  company                362 non-null    object        
 11  publication_date       362 non-null    dat

## Описание вакансии - description

In [31]:
# Вытащим описание вакансии
df_sorted['description'] = df_sorted['html'].apply(lambda x: BeautifulSoup(x).find('div', {'id':'job-details'}).text.strip())

## Хард скилы - skills

In [32]:
# Какие хард скилы требуются дата аналитику - тащим это из описания вакансии, то есть из столбца description
skills = (['datahub', 'api', 'github', 'google analytics', 'adobe analytics', 'ibm coremetrics', 'omniture'
            'gitlab', 'erwin', 'hadoop', 'spark', 'hive'
           'databricks', 'aws', 'gcp', 'azure','excel',
            'redshift', 'bigquery', 'snowflake',  'hana'
            'grafana', 'kantar', 'spss', 
           'asana', 'basecamp', 'jira', 'dbeaver','trello', 'miro', 'salesforce', 
           'rapidminer', 'thoughtspot',  'power point',  'docker', 'jenkins','integrate.io', 'talend', 'apache nifi','aws glue','pentaho','google data flow',
             'azure data factory','xplenty','skyvia','iri voracity','xtract.io','dataddo', 'ssis',
             'hevo data','informatica','oracle data integrator','k2view','cdata sync','querysurge', 
             'rivery', 'dbconvert', 'alooma', 'stitch', 'fivetran', 'matillion','streamsets','blendo',
             'iri voracity','logstash', 'etleap', 'singer', 'apache camel','actian', 'airflow', 'luidgi', 'datastage',
           'python', 'vba', 'scala', ' r ', 'java script', 'julia', 'sql', 'matlab', 'java', 'html', 'c++', 'sas',
           'data studio', 'tableau', 'looker', 'powerbi', 'cognos', 'microstrategy', 'spotfire',
             'sap business objects','microsoft sql server', 'oracle business intelligence', 'yellowfin',
             'webfocus','sas visual analytics', 'targit', 'izenda',  'sisense', 'statsbot', 'panorama', 'inetsoft',
             'birst', 'domo', 'metabase', 'redash', 'power bi', 'alteryx', 'dataiku', 'qlik sense', 'qlikview'
          ]) 
def get_skills(cell):
    list_skills = []
    for skill in skills:
        if skill in cell.lower().replace('powerbi', 'power bi'):
            list_skills.append(skill)
    return list_skills
df_sorted['skills'] = df_sorted.description.apply(get_skills)
df_sorted.sample(5)

,html,title,link,geo,city,country,latitude,longitude,candidats,employment,company,publication_date,size_field,company_size,company_size_category,field_of_activity,description,skills
134,"\n <div>\n <div class=""\n jobs-deta...",Data Analyst,https://linkedin.com/jobs/view/3258112915/?alt...,"Haydock, England, United Kingdom",Haydock,United Kingdom,53.467195,-2.661806,1,On-site,Speedy Services,2022-09-07 15:00:00,"1,001-5,000 employees · Retail Office Equipment",1001-5000,large,Retail Office Equipment,Speedy are the UK’s leading hire provider with...,"[excel, sql]"
158,"\n <div>\n <div class=""\n jobs-deta...",Internet Analyst,https://linkedin.com/jobs/view/3249279868/?alt...,"Trenčín, Trenciansky, Slovakia",Trenčín,Slovakia,48.892358,18.039372,0,Remote,TELUS International AI Data Solutions,2022-09-07 06:00:00,"10,001+ employees · IT Services and IT Consulting",10001+,large,IT Services and IT Consulting,TELUS International AI-Data Solutions partners...,[]
343,"\n <div>\n <div class=""\n jobs-deta...","Internet Analyst | Florence, Italy Metropolita...",https://linkedin.com/jobs/view/3247842966/?alt...,"Florence, Italy Metropolitan Area",Florence,Italy,43.808236,11.293806,7,Remote,Appen,2022-09-05 17:00:00,"501-1,000 employees · IT Services and IT Consu...",501-1000,small,IT Services and IT Consulting,Good day!\n\nAppen is seeking candidates for E...,"[excel, ssis]"
19,"\n <div>\n <div class=""\n jobs-deta...",Data Analyst,https://linkedin.com/jobs/view/3257251472/?alt...,"Chełm, Lubelskie, Poland",Chełm,Poland,51.135588,23.493937,5,Remote,TELUS International AI Data Solutions,2022-09-07 09:00:00,"10,001+ employees · IT Services and IT Consulting",10001+,large,IT Services and IT Consulting,TELUS International AI-Data Solutions partners...,[]
108,"\n <div>\n <div class=""\n jobs-deta...",STAGE - DATA ANALYST MARKETING F/H,https://linkedin.com/jobs/view/3249413715/?alt...,"Bordeaux, Nouvelle-Aquitaine, France",Bordeaux,France,44.841225,-0.580036,6,Hybrid,Cdiscount,2022-09-02 17:00:00,"1,001-5,000 employees · Technology, Informatio...",1001-5000,large,"Technology, Information and Internet",REJOIGNEZ UN COLLECTIF ANIME PAR LE GOÛT DU DE...,"[excel, python, sql, tableau]"


In [33]:
# Выгрузим скилы в длинную таблицу
df = df_sorted.explode('skills')

# Сохраним таблицу для Tableau
df[['candidats', 'title', 'city', 'country', 'skills', 'link']].to_csv('ready_ready.csv', index = False)